## Rotating IP address with Bright Data and Python library Selenium

Reference: 
1. How to rotate proxies in Selenium Python? [5 Easy Steps] https://techjury.net/blog/how-to-rotate-proxies-in-selenium-python/
2. How to Use Selenium Wire in 2024 https://www.zenrows.com/blog/selenium-wire
3. Scrape shopee using request is getting error type 2 https://stackoverflow.com/questions/75106834/scrape-shopee-using-request-is-getting-error-type-2

In [1]:
%pip install selenium
%pip install botasaurus
%pip install webdriver-manager
%pip install watermark # to inspect what python package imported in this jupyter notebook

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [2]:
import os
import random
from selenium import webdriver
from botasaurus_proxy_authentication import add_proxy_options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.by import By
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
url = "https://httpbin.co/ip"

proxy_hostname = os.getenv("PROXY_HOSTNAME")
proxy_username = os.getenv("PROXY_USERNAME")
proxy_password = os.getenv("PROXY_PASSWORD")

In [4]:
class ProxyServer:
    def __init__(self, proxy_username, proxy_password, proxy_hostname):
        self.proxy_username = proxy_username
        self.proxy_password = proxy_password
        self.proxy_hostname = proxy_hostname

    def __call__(self):
        rand_num = random.randint(1, 9999)
        # Reference: https://docs.brightdata.com/api-reference/proxy/rotate_ips
        ## Add -session parameter to this proxy to change IP address every requests is sent
        proxy = f"{self.proxy_username}-session-rand{rand_num}:{self.proxy_password}@{self.proxy_hostname}"
        return f"http://{proxy}"


proxy_servers = ProxyServer(proxy_username, proxy_password, proxy_hostname)
# call proxy_servers() to register a new IP proxy
# >> proxies = proxy_servers() 

In [5]:
import json
import time


for idx in range(1, 11):
    options = Options()
    # add bright data's proxies
    add_proxy_options(options, proxy_servers()) 
    
    # options.add_argument("--headless=new") # uncomment this code if you want to set to headless mode (= no UI)
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                            options=options)
    driver.get(url=url)
    time.sleep(3)
    content = driver.page_source
    time.sleep(1)
    content = driver.find_element(by=By.TAG_NAME, value='pre').text
    parsed_json = json.loads(content)
    print("IP location metadata:", parsed_json)
    driver.quit()

IP location metadata: {'method': 'GET', 'ip': '206.204.50.94', 'country': 'US', 'timezone': 'America/Chicago', 'continent': 'NA'}
IP location metadata: {'method': 'GET', 'ip': '91.132.38.119', 'country': 'DK', 'timezone': 'Europe/Copenhagen', 'continent': 'EU'}
IP location metadata: {'method': 'GET', 'ip': '45.143.106.105', 'country': 'AM', 'region': 'Yerevan', 'city': 'Yerevan', 'timezone': 'Asia/Yerevan', 'continent': 'AS'}
IP location metadata: {'method': 'GET', 'ip': '158.46.187.57', 'country': 'LT', 'region': 'Vilnius', 'city': 'Vilnius', 'timezone': 'Europe/Vilnius', 'continent': 'EU'}
IP location metadata: {'method': 'GET', 'ip': '176.53.221.75', 'country': 'RU', 'timezone': 'Europe/Moscow', 'continent': 'EU'}
IP location metadata: {'method': 'GET', 'ip': '193.187.113.111', 'country': 'MY', 'timezone': 'Asia/Kuching', 'continent': 'AS'}
IP location metadata: {'method': 'GET', 'ip': '158.46.167.163', 'country': 'TH', 'timezone': 'Asia/Bangkok', 'continent': 'AS'}
IP location meta

## Computing environment

In [7]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z

Last updated: 2024-03-03T21:40:40.672927+08:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.22.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 12, GenuineIntel
CPU cores   : 8
Architecture: 64bit

selenium: 4.5.0
json    : 2.0.9

Last updated: Sun Mar 03 2024 21:40:40Malay Peninsula Standard Time

